In [269]:
import sys
import numpy as np
from functools import reduce
import sounddevice as sd

from pam import SoundCommunication

In [270]:
import unireedsolomon.rs as rs

In [271]:
# sampling rate
FS = 44100

In [272]:
def prepare_chunks(text, chunk_size):
    text_size = len(text)
    
    N_chunks = text_size//chunk_size if text_size%chunk_size == 0 else text_size//chunk_size + 1
    
    chunks = []
    for i in range(N_chunks-1):
        l = i*chunk_size
        r = i*chunk_size + chunk_size
        
        chunks.append(text[l:r])
    chunks.append(text[chunk_size*(N_chunks-1):len(text)])
    return chunks

In [273]:
def encode_rs(msg):
    rs_coder = rs.RSCoder(255, 185)
    return rs_coder.encode(msg)

In [275]:
def text2bits(text):
    bytes_ = bytes(text, 'utf-8')

    bits = str()
    for b in bytes_:
        res = bin(b)[2:]
        res2 = '0'*(0 if len(res)%8 == 0 else 8 - len(res)%8) + res
      
        assert len(res2) == 8
        bits += res2
    return bits

In [276]:
def pad(W, padding=0.5):
    return np.hstack((np.zeros(int(FS*padding)), W, np.zeros(int(FS*padding))))

In [281]:
def transmit(text):
    text_rs = encode_rs(text)
    text_bits = text2bits(text_rs)
    print(len(text_bits))
    comm = SoundCommunication(FS, 200, 1000, 2000, msgSymLen=len(text_bits))
    S = comm.send(text_bits)
    sd.play(pad(S), FS)

In [282]:
merge_lines = lambda text, l: l if not text else text + l

with open('text.txt') as f:
    lines = f.readlines()
    text = reduce(merge_lines, lines, '')

transmit(text)

2328


In [283]:
sd.stop()